# BERT를 사용한 의료 문서 분류

2022학년도 2학기 [자연어처리를 이용한 의료정보추출 및 분석] 기말과제



---


## 과제 개요

### 목표
##### 영어로 된 의료 문서 데이터가 **수술기록지인지 아닌지**를 자동으로 분류하고자 한다.

### 방법
##### 사전 훈련된(pre-trained) BERT 모델을 위의 분류 목적에 맞게 미세 조정(fine-tuning)한다.

### 데이터셋
##### https://github.com/socd06/medical-nlp



---

## 데이터셋 준비하기
*   주의: Google Colab 클라우트 환경에는 pandas 라이브러리가 이미 설치되어 있으나, 개인 컴퓨터 로컬 환경에서는 해당 라이브러리를 수동으로 설치해야 한다. 아래의 코드로 설치가 가능하다. 



```
!pip install -U pandas
```


### 데이터셋 파일 내려받기
##### 이제 pandas 라이브러리의 read_csv() 함수를 사용하여 데이터 파일의 URL을 직접 읽어오자.
##### read_csv() 함수는 CSV 파일을 읽어서 표의 형식으로 이루어진 데이터프레임 자료형으로 저장한다.

In [2]:
import pandas as pd

data_url = "https://raw.githubusercontent.com/socd06/medical-nlp/master/data/X.csv"
df = pd.read_csv(data_url)
df.head()

,label,description,text
0,4,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr..."
1,4,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb..."
2,4,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ..."
3,3,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
4,3,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...


##### 위의 출력 결과를 통해 데이터셋이 레이블(label), 제목(description), 내용(text), 세 개의 속성으로 이루어져 있다는 것을 알 수 있다.

##### 다음으로는 레이블의 값이 몇 가지인지를 알아보기 위해 df['label']로 행을 선택하고 value_counts() 메소드로 행의 값들을 세어 보자.

In [3]:
df['label'].value_counts()

1    1640
2    1228
3    1149
4     982
Name: label, dtype: int64

각 레이블의 의미는 다음과 같다.


1.   Surgery
2.   Medical Records
3.   Internal Medicine
4.   Other

### 데이터셋 가공하기

#### 레이블 선별

##### 이 과제에서는 4가지 레이블 중 1과 2만 사용하려고 한다.

##### df['label']로 행을 선택하고 isin() 메소드를 이용하여 값이 1, 2에 속하는 것들만 추려낸 뒤 결과를 df12라는 새로운 데이터프레임으로 저장하자.

In [4]:
df12 = df[df['label'].isin((1,2))]
df12

,label,description,text
18,1,Vasectomy - 4,"PROCEDURE: , Elective male sterilization via b..."
20,1,Whole Body Radionuclide Bone Scan,"INDICATION:, Prostate Cancer.,TECHNIQUE:, 3...."
22,1,Vasectomy - 1,"DESCRIPTION:, The patient was placed in the s..."
23,1,Vasectomy,"PREOPERATIVE DIAGNOSIS: , Voluntary sterility...."
24,1,Urology Consut - 1,"CHIEF COMPLAINT:,nan"
...,...,...,...
4592,2,Abnormal Echocardiogram,"REASON FOR CONSULTATION: ,Abnormal echocardio..."
4593,2,Adjustment Disorder & Encopresis,"REASON FOR REFERRAL:, The patient was referre..."
4594,2,Acute Inferior Myocardial Infarction,"CHIEF COMPLAINT: , Chest pain.,HISTORY OF PRES..."
4595,2,A 5-month-old boy with cough,"CHIEF COMPLAINT:, A 5-month-old boy with coug..."


#### 레이블 값 변경

##### 현재 사용할 데이터셋에서 레이블 값은 1과 2로 이루어져 있다. 그런데 레이블은 0부터 시작하는 것이 일반적이다. 레이블 값 2를 0으로 바꾸어 주자.

##### 데이터프레임 df12의 값을 복제하여 df10 데이터프레임을 만들고, df10.loc[행_인덱스, 열_인덱스] 로 값이 2인 셀을 선택한 뒤 0의 값을 할당할 수 있다.

In [5]:
df10 = df12.copy()
df10.loc[df10['label']==2, 'label'] = 0
df10

,label,description,text
18,1,Vasectomy - 4,"PROCEDURE: , Elective male sterilization via b..."
20,1,Whole Body Radionuclide Bone Scan,"INDICATION:, Prostate Cancer.,TECHNIQUE:, 3...."
22,1,Vasectomy - 1,"DESCRIPTION:, The patient was placed in the s..."
23,1,Vasectomy,"PREOPERATIVE DIAGNOSIS: , Voluntary sterility...."
24,1,Urology Consut - 1,"CHIEF COMPLAINT:,nan"
...,...,...,...
4592,0,Abnormal Echocardiogram,"REASON FOR CONSULTATION: ,Abnormal echocardio..."
4593,0,Adjustment Disorder & Encopresis,"REASON FOR REFERRAL:, The patient was referre..."
4594,0,Acute Inferior Myocardial Infarction,"CHIEF COMPLAINT: , Chest pain.,HISTORY OF PRES..."
4595,0,A 5-month-old boy with cough,"CHIEF COMPLAINT:, A 5-month-old boy with coug..."


#### 데이터 순서 섞기

##### 데이터셋을 살펴보면 레이블 1에 속하는 데이터가 먼저 나열되고 그 다음에 0이 이어지는 것을 확인할 수 있다. 이 경우 원활한 학습이 이뤄지지 않을 수 있다.

##### 데이터의 순서를 섞어서 골고루 분포하도록 만들어 주자. 데이터프레임에서 sample() 메소드를 사용하면 된다. random_state=0 옵션을 추가하면 결과를 재현할 수 있다.

In [6]:
df10_shuffled = df10.sample(frac=1, random_state=0)
df10_shuffled

,label,description,text
3832,0,Hematuria - ER Visit,"HISTORY OF PRESENT ILLNESS:, The patient is a..."
551,1,Metastasectomy & Bronchoscopy,"PREOPERATIVE DIAGNOSIS: ,Metastatic renal cel..."
4370,0,Gen Med Consult - 52,"CHIEF COMPLAINT: , Anxiety, alcohol abuse, and..."
4299,0,Lesions - Adrenal and Pancreatic,"CHIEF COMPLAINT: , Both pancreatic and left ad..."
556,1,Mass Excision - Foot,"PREOPERATIVE DIAGNOSIS: , Soft tissue mass, ri..."
...,...,...,...
783,1,Foreign Body Removal - Thigh,"PREPROCEDURE DIAGNOSIS:, Foreign body of the ..."
855,1,Endoscopy With Biopsy,"PROCEDURE:, Upper endoscopy with biopsy.,PROC..."
1712,0,CT Abdomen & Pelvis - 10,CT ABDOMEN WITHOUT CONTRAST AND CT PELVIS WITH...
4336,0,Head Trauma,"CC: ,Depressed mental status.,HX: ,29y/o femal..."


##### 이것으로 데이터프레임 단계에서 데이터를 가공하는 과정을 마쳤다.

## HuggingFace 라이브러리 설치하기

##### 이제 BERT 모델을 사용할 수 있도록 아래의 두 가지 라이브러리를 설치한다.

*   datasets : pandas 데이터프레임을 BERT 훈련에 사용할 수 있는 객체로 변환한다.
*   transformers : BERT 모델과 토크나이저를 사용할 수 있도록 한다.

```
!pip install -U datasets transformers
```





### 훈련집합-시험집합 분할



*   참조 : https://huggingface.co/docs/datasets/process

##### 이제 datasets 라이브러리를 사용하여 데이터를 BERT 훈련에 사용할 수 있는 객체로 변환한다.

##### 먼저 from_pandas() 메소드를 통해 df10_shuffled 로 부터 dataset 객체를 만든다. BERT 등 트랜스포머 모델을 훈련하고 평가할 때는 이 dataset 객체를 사용한다.

##### 이때 훈련에 사용할 데이터셋과 평가에 사용할 데이터셋이 서로 중복되어선 안 된다. dataset 객체에 train_test_split() 메소드를 사용하여 train, test 셋을 분할한다.

##### 옵션으로 test_size = 0.1 을 주어 90%를 train set, 10%를 test set으로 나누고, seed=0 을 주어 결과를 재현한다.

In [7]:
!pip install -U datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [8]:
from datasets import Dataset

# 데이터셋 초기화
dataset = Dataset.from_pandas(df10_shuffled, preserve_index=True)

# train / test 셋으로 분할
dataset = dataset.train_test_split(test_size=0.1, seed=0)

dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'description', 'text', '__index_level_0__'],
        num_rows: 2581
    })
    test: Dataset({
        features: ['label', 'description', 'text', '__index_level_0__'],
        num_rows: 287
    })
})

##### 위의 출력결과를 보면 train이 2581개, test가 287개의 데이터로 이루어져 있음을 알 수 있다.

##### train set과 test set 모두 원래의 데이터프레임과 동일한 'label', 'description', 'text' 세 개의 속성을 가지고 있으며, 추가로 인덱스 속성을 가지고 있다.

##### 예를 들어 dataset['test']의 42번 인덱스에 해당하는 값이 어떻게 생겼는지 살펴보자.

In [9]:
from pprint import pprint
pprint(dataset['test'][42])

{'__index_level_0__': 159,
 'description': ' Circumcision - 7 ',
 'label': 1,
 'text': 'PROCEDURE:,  Circumcision.,ANESTHESIA: , EMLA.,FINDINGS: , Normal '
         'penis.  The foreskin was normal in appearance and measured 1.6 cm.  '
         'There was no bleeding at the circumcision site.,PROCEDURE:,  Patient '
         'was placed on the circumcision restraint board.  EMLA had been '
         'applied approximately 90 minutes before.  A time-out was completed '
         'satisfactorily per protocol.  The area was prepped with Betadine.  '
         'The foreskin was grasped with sterile clamps and was dissected away '
         'from the corona and the glans penis with blunt dissection.  A Mogen '
         'clamp was applied to the cervix.  The excess foreskin was excised '
         'with the scalpel.  The clamp was removed.  At this point, the '
         'procedure was terminated.  Sterile Vaseline and gauze was applied to '
         'the glans penis.  There were no complications. 

##### 인덱스가 159, 제목이 'Circumcision-7', 레이블이 1(수술기록지)임을 알 수 있다.

## 텍스트 전처리

### BERT 토크나이저 가져오기

*   참조 : https://huggingface.co/docs/datasets/nlp_process

##### 위의 문서를 참조하여, 구글에서 공개한 BERT-BASE 모델의 토크나이저를 가져와보자. transfermers 라이브러리에서 제공하는 AutoTokenizer 자료형을 사용하고, model_path 값으로 BERT-BASE 모델의 이름을 넣는다.

##### BERT-BASE 모델에 대한 자세한 정보는 다음 URL에서 얻을 수 있다. https://huggingface.co/bert-base-uncased



In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# BERT-BASE
model_path = "bert-base-uncased"

# BERT 토크나이저 가져오기
tokenizer = AutoTokenizer.from_pretrained(model_path)

# BERT 모델 가져오기
model = AutoModelForSequenceClassification.from_pretrained(model_path)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

##### 만일 BERT-BASE 외의 다른 모델을 사용하고 싶은 경우 https://huggingface.co 에서 다양한 모델을 검색할 수 있다.

### 텍스트 토크나이징

##### 토크나이저를 사용하는 목적은 하나의 문자열로 이루어진 문장을 여러 개의 subword 토큰으로 분리하는 것이다. 토른은 대략 단어 혹은 그보다 작은 단위로 이해할 수 있다.

> 원래 문장 : "나는 사과를 먹었다."
>
> 토크나이징 결과 : "나", "##는", "사과", "##를", "먹", "##었", "##다.", "##"

##### BERT 모델은 문장을 subword 토큰으로 분리한 뒤 각 토큰마다 토큰의 의미를 표상하는 임베딩 벡터를 할당하고 훈련시킨다.

*   참조 (1) : https://huggingface.co/docs/datasets/nlp_process
*   참조 (2) : https://huggingface.co/docs/transformers/v4.21.3/en/pad_truncation

##### 이제 위의 두 문서를 참조하여 토큰 분리를 실행하는 코드를 작성해 보자.

*   주의 : 토큰의 개수는 문장(혹은 문서)마다 달라진다. 하지만 BERT 모델에서 다양한 문장을 한꺼번에 처리하기 위해서는 문장의 길이가 통일되어야 한다. 그러므로 BERT 모델의 하이퍼파라미터인 최대 길이(max_position_embedding)에 맞추어 긴 문장은 뒷부분을 잘라주고, 짧은 문장의 뒤에는 [PAD] 토큰을 추가해주어야 한다. 이를 위해 tokenizer 함수에는 padding과 truncation 옵션이 존재한다.

##### 아래에서 정의한 함수는 문장(examples['text'])을 받아서 토큰으로 분리해주고, BERT-BASE 모델에 설정된 최대 길이(model.config.max_position_embeddings)에 맞게 padding과 truncation을 적용하여 문장의 길이(토큰 개수)를 통일해준다.



In [11]:
def tokenize_many(examples):
  return tokenizer(
      examples['text'],  # 토큰 분리의 대상이 되는 문장
      max_length=model.config.max_position_embeddings,  # BERT 모델에서 미리 설정된 최대 길이
      padding='max_length',  # 짧은 문장은 패딩하기
      truncation=True,  # 긴 문장은 자르기
  )

##### 지금 만든 함수는 dataset.map() 메소드를 이용하여 dataset 객체의 모든 문서에 적용할 수 있다.

In [12]:
dataset = dataset.map(tokenize_many, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

##### 토큰 분리 결과를 살펴보자.

In [13]:
print(dataset['test'][42].keys())

dict_keys(['label', 'description', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'])


##### 앞서 살펴본 바로는 데이터셋의 각 문자가 label / description / text / index_level_0 네 개의 키를 가졌었는데, 토큰 분리 결과 'input_ids', 'token_type_ids', 'attention_mask' 세 개의 키가 추가되었음을 알 수 있다.

##### BERT 모델의 입력으로 필수적으로 들어가야 할 내용은 'input_ids' 키다. 이 키의 값으로는 정수로 이루어진 리스트가 나온다. 이 정수값들은 subword 토큰에 대응하는 아이디를 말한다.

##### 즉 실험집합 42번째 문서는 하나의 문자열로부터 101번 토큰( [CLS] ), 7709번 토큰, 1024번 토큰, ... 등으로 분리되었다.

In [14]:
print(dataset['test'][42]['input_ids'])

[101, 7709, 1024, 1010, 25022, 11890, 2819, 28472, 1012, 1010, 2019, 25344, 1024, 1010, 7861, 2721, 1012, 1010, 9556, 1024, 1010, 3671, 19085, 1012, 1996, 18921, 29334, 2001, 3671, 1999, 3311, 1998, 7594, 1015, 1012, 1020, 4642, 1012, 2045, 2001, 2053, 9524, 2012, 1996, 25022, 11890, 2819, 28472, 2609, 1012, 1010, 7709, 1024, 1010, 5776, 2001, 2872, 2006, 1996, 25022, 11890, 2819, 28472, 19355, 2604, 1012, 7861, 2721, 2018, 2042, 4162, 3155, 3938, 2781, 2077, 1012, 1037, 2051, 1011, 2041, 2001, 2949, 2938, 2483, 7011, 16761, 6588, 2566, 8778, 1012, 1996, 2181, 2001, 17463, 5669, 2007, 8247, 10672, 1012, 1996, 18921, 29334, 2001, 15517, 2007, 25403, 18856, 25167, 1998, 2001, 4487, 11393, 10985, 2185, 2013, 1996, 21887, 1998, 1996, 1043, 5802, 2015, 19085, 2007, 14969, 4487, 11393, 7542, 1012, 1037, 9587, 6914, 18856, 16613, 2001, 4162, 2000, 1996, 8292, 2099, 5737, 2595, 1012, 1996, 9987, 18921, 29334, 2001, 4654, 18380, 2094, 2007, 1996, 21065, 2884, 1012, 1996, 18856, 16613, 2001, 371

##### 지금까지 텍스트로 이루어진 문서를 읽고 BERT 모델의 입력 포맷에 맞는 토큰 아이디의 배열로 변환하는 과정을 실습하였다. 다음 시각에는 이 데이터를 사용하여 BERT 모델을 파인튜닝하는 방법을 알아볼 것이다.